In [1]:
import pandas as pd
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima_model import ARMA
from scipy import stats
from statsmodels.tsa.seasonal import seasonal_decompose

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)
%matplotlib inline
#%matplotlib notebook
sns.set(rc={'figure.figsize':(11.7,8.27)})
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime,timedelta
from itertools import product
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

In [4]:
# data_16_18 = pd.read_excel(r'siv_2016-18.xlsx',sheetname='2016-18')
# data_19 = pd.read_excel(r'siv_2016-18.xlsx',sheetname='2019')

# data_16_18.to_pickle('data_16_18.pkl')

# data_19.to_pickle('data_19.pkl')

In [ ]:
data_16_18.drop(['Fiscal year/period', 'Date', 'year','key'],axis=1,inplace=True)
data_16_18.columns= ['Distributor', 'matCode', 'siv_vol', 'timestamp']

data_16_18.head()

data_16_18['timestamp'] = data_16_18['timestamp'].dt.strftime("%Y-%m-%d")

piv_data = data_16_18.pivot_table(index=['Distributor','matCode'],columns='timestamp',aggfunc='sum',\
                            values='siv_vol',fill_value=0,dropna=False,margins=True).reset_index()
piv_data.fillna(0,inplace=True)

In [42]:
data_19_sel = data_19.loc[:,['Distributor','Material','Month of Effective Date','SIV']]
data_19_sel['Month of Effective Date'] = data_19_sel['Month of Effective Date'].dt.strftime("%Y-%m-%d")

piv_19 = data_19_sel.pivot_table(index=['Distributor','Material'],columns='Month of Effective Date',aggfunc='sum',\
                            values='SIV',fill_value=0,dropna=False,margins=True).reset_index()
piv_19.fillna(0,inplace=True)

In [46]:
piv_19.shape

(124101, 17)

In [47]:
piv_data.shape

(69105, 39)

In [48]:
piv_data.to_excel(r'siv_trend_16_18.xlsx',index=False)

In [49]:
df_16_18 = pd.read_pickle('data_16_18.pkl')

df_16_18.drop(['Fiscal year/period', 'Date', 'year','key'],axis=1,inplace=True)
df_16_18.columns= ['Distributor', 'matCode', 'siv_vol', 'timestamp']
df_16_18.set_index('timestamp',inplace=True)

In [54]:
df_q = df_16_18.loc[:,['Distributor', 'siv_vol']]
df_q.groupby('Distributor').resample('Q').sum().reset_index()
df_q.groupby('Distributor').resample('Q').sum().reset_index()
df_16_18.groupby('Distributor')['siv_vol'].resample('Q').sum().reset_index()
df_16_18.groupby('matCode')['siv_vol'].resample('Q').sum().reset_index()

In [61]:
df_16_18.head()

,Distributor,matCode,siv_vol
timestamp,,,
2016-01-01,ANAND MOTORS,132454,11.92
2016-01-01,ANAND MOTORS,135988,96.04
2016-01-01,ANAND MOTORS,136059,24.01
2016-01-01,ANAND MOTORS,136106,24.01
2016-01-01,ANAND MOTORS,139284,"1,040.02"


In [62]:
data_19_sel.columns = ['Distributor', 'matCode', 'timestamp', 'siv_vol']

data_19_sel.set_index('timestamp',inplace=True)

In [64]:
data_19_sel.head()

,Distributor,matCode,siv_vol
timestamp,,,
2019-01-01,A K ENTERPRISES,123331,0.00
2019-02-01,A K ENTERPRISES,123331,0.00
2019-03-01,A K ENTERPRISES,123331,0.00
2019-04-01,A K ENTERPRISES,123331,0.00
2019-05-01,A K ENTERPRISES,123331,0.00


In [88]:
df_all = pd.concat([df_16_18,data_19_sel],axis=0)

In [69]:
df_all.shape

(367618, 3)

In [81]:
df_2 = df_all.reset_index()

df_2['timestamp'] = df_2['timestamp'].dt.strftime("%Y-%m-%d")

piv_data_all = df_2.reset_index().pivot_table(index=['Distributor','matCode'],columns='timestamp',aggfunc='sum',\
                            values='siv_vol',fill_value=0,dropna=False,margins=True).reset_index()
piv_data_all.fillna(0,inplace=True)

In [82]:
piv_data_all.head()

timestamp,Distributor,matCode,2016-01-01,2016-02-01,2016-03-01,2016-04-01,2016-05-01,2016-06-01,2016-07-01,2016-08-01,2016-09-01,2016-10-01,2016-11-01,2016-12-01,2017-01-01,2017-02-01,2017-03-01,2017-04-01,2017-05-01,2017-06-01,2017-07-01,2017-08-01,2017-09-01,2017-10-01,2017-11-01,2017-12-01,2018-01-01,2018-02-01,2018-03-01,2018-04-01,2018-05-01,2018-06-01,2018-07-01,2018-08-01,2018-09-01,2018-10-01,2018-11-01,2018-12-01,2019-01-01,2019-02-01,2019-03-01,2019-04-01,2019-05-01,2019-06-01,2019-07-01,2019-08-01,2019-09-01,2019-10-01,2019-11-01,2019-12-01,2020-01-01,2020-02-01,All
0,A K ENTERPRISES,120082,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,A K ENTERPRISES,120088,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,A K ENTERPRISES,120089,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,A K ENTERPRISES,120182,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,A K ENTERPRISES,120229,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [83]:
piv_data_all.shape

(125686, 53)

In [84]:
piv_data_all.columns

Index(['Distributor', 'matCode', '2016-01-01', '2016-02-01', '2016-03-01',
       '2016-04-01', '2016-05-01', '2016-06-01', '2016-07-01', '2016-08-01',
       '2016-09-01', '2016-10-01', '2016-11-01', '2016-12-01', '2017-01-01',
       '2017-02-01', '2017-03-01', '2017-04-01', '2017-05-01', '2017-06-01',
       '2017-07-01', '2017-08-01', '2017-09-01', '2017-10-01', '2017-11-01',
       '2017-12-01', '2018-01-01', '2018-02-01', '2018-03-01', '2018-04-01',
       '2018-05-01', '2018-06-01', '2018-07-01', '2018-08-01', '2018-09-01',
       '2018-10-01', '2018-11-01', '2018-12-01', '2019-01-01', '2019-02-01',
       '2019-03-01', '2019-04-01', '2019-05-01', '2019-06-01', '2019-07-01',
       '2019-08-01', '2019-09-01', '2019-10-01', '2019-11-01', '2019-12-01',
       '2020-01-01', '2020-02-01', 'All'],
      dtype='object', name='timestamp')

In [85]:
df_all.head()

,Distributor,matCode,siv_vol
timestamp,,,
2016-01-01 00:00:00,ANAND MOTORS,132454,11.92
2016-01-01 00:00:00,ANAND MOTORS,135988,96.04
2016-01-01 00:00:00,ANAND MOTORS,136059,24.01
2016-01-01 00:00:00,ANAND MOTORS,136106,24.01
2016-01-01 00:00:00,ANAND MOTORS,139284,"1,040.02"


In [89]:
qtr_distributor_demand = df_all.groupby(['Distributor'])['siv_vol'].resample('Q').sum()

TypeError: Cannot compare type 'Timestamp' with type 'str'

In [ ]:
df_all = pd.concat([df_16_18,data_19_sel],axis=0)

In [93]:
data_19_sel.index.dtype

dtype('O')

In [103]:
df_19 = pd.read_pickle('data_19.pkl')

df_19_sel = df_19.loc[:,['Distributor','Material','Month of Effective Date','SIV']]

df_19_sel.columns = ['Distributor', 'matCode', 'timestamp', 'siv_vol']
df_19_sel.set_index('timestamp',inplace=True)

In [105]:
df_all = pd.concat([df_16_18,data_19_sel],axis=0)

In [106]:
df_all.index.dtype

dtype('O')

In [107]:
df_all.index = pd.to_datetime(df_all.index)

In [110]:
qtr_distributor_demand = df_all.groupby(['Distributor'])['siv_vol'].resample('Q').sum().reset_index()

In [112]:
qtr_distributor_demand.head(10)

,Distributor,timestamp,siv_vol
0,A K ENTERPRISES,2017-12-31,"3,630.45"
1,A K ENTERPRISES,2018-03-31,"8,523.84"
2,A K ENTERPRISES,2018-06-30,"13,982.27"
3,A K ENTERPRISES,2018-09-30,"9,778.11"
4,A K ENTERPRISES,2018-12-31,"20,058.04"
5,A K ENTERPRISES,2019-03-31,"16,123.72"
6,A K ENTERPRISES,2019-06-30,"20,293.50"
7,A K ENTERPRISES,2019-09-30,"24,406.70"
8,A K ENTERPRISES,2019-12-31,"20,741.74"
9,A K ENTERPRISES,2020-03-31,"12,691.43"
